# Plant Seedling Classification

<img src="https://www.gardeningknowhow.com/wp-content/uploads/2019/10/seedling.jpg" height=500 width=600>

# 0. Intro

**Task type:** Classification

**ML algorithms used:** Convolutional neural network

**Other features:** Visualizing the filters used by the CNN

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt

import cv2

# 1. Image importing

In [ ]:
image_size = 256
batch_size = 32

**I will load the images using ImageDataGeneretor class and explicitly indicate the needed parameters (rescaling, flipping, validation splitting.**

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    #rotation_range=20, # You can uncomment these parameters to make you generator rotate & flip the images to put the train model in stricter conditions.
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

**Train set**

In [ ]:
train_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=1
                                                )

**Validation set**

In [ ]:
val_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=1
                                                )

In [ ]:
unique, counts = np.unique(train_gen.classes, return_counts=True)
dict1 = dict(zip(train_gen.class_indices, counts))

keys = dict1.keys()
values = dict1.values()

plt.xticks(rotation='vertical')
bar = plt.bar(keys, values)

**The train dataset is quite balanced.**

**Let's visualize a portion of images to make sure they're correctly loaded.**

In [ ]:
x,y = next(train_gen)

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def show_grid(image_list, nrows, ncols, label_list=None, show_labels=False, figsize=(10,10)):

    fig = plt.figure(None, figsize,frameon=False)
    grid = ImageGrid(fig, 111, 
                     nrows_ncols=(nrows, ncols),  
                     axes_pad=0.2, 
                     share_all=True,
                     )
    for i in range(nrows*ncols):
        ax = grid[i]
        ax.imshow(image_list[i],cmap='Greys_r')
        ax.axis('off')

In [ ]:
show_grid(x,2,4,show_labels=True,figsize=(10,10))

**The images are nicely loaded and do not have any rotation and distortion.**

# 2. Model building

**Quick description, before heading on:**

**Model type:** Sequential

**Layers used:**

    0. InputLayer
    1. Conv2D (64, 128 filters)
    2. MaxPool2D
    3. GlobalMaxPool2D
    4. Batch Normalization
    5. Flatten
    6. Dropout
    6. Dense
    
**Input size:** 256 x 256 x 3 (size x colors)

**Pool size:** 2 x 2 (for MaxPool2D)

**Kernel size:** 3 x 3 (for Conv2D)

In [ ]:
model = tf.keras.models.Sequential()

# Input layer
# Can be omitted, you can specify the input_shape in other layers
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))

# Here we add a 2D Convolution layer
# Check https://keras.io/api/layers/convolution_layers/convolution2d/ for more info
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

# Max Pool layer 
# It downsmaples the input representetion within the pool_size size
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
# The layer normalizes its output using the mean and standard deviation of the current batch of inputs.
model.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
model.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
model.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Global Max Pool layer
model.add(tf.keras.layers.GlobalMaxPool2D())

# Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))

# Dropout
# is used to nullify the outputs that are very close to zero and thus can cause overfitting.
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))

# Normalization layer
model.add(tf.keras.layers.BatchNormalization())

#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax')) # = 12 predicted classes

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# You can save the best model to the checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_classifier.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history = model.fit(train_gen,
          epochs=20, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint],
          verbose = 1
)

**Learning curves vs epoch graph**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(list(range(1,21)))
plt.ylim([0, 1])
plt.legend(loc='lower right')

# 3. Visualizing

**Here we will check the predictions made by our model as well as visualize how the model filter one of the images taken from the dataset.**

**Let's first check the prediction power.**

In [ ]:
maize = cv2.imread('../input/plant-seedlings-classification/train/Maize/6e9ff31e7.png')

In [ ]:
plt.imshow(maize)

**We need to preprocess the image before passing it on to the model: resize + expand_dims.**

In [ ]:
maize = cv2.resize(maize, (256,256))

In [ ]:
maize_batch = np.expand_dims(maize, axis=0)

In [ ]:
conv_maize = model.predict(maize_batch)

In [ ]:
conv_maize.shape

In [ ]:
def visualize(maize_batch):
    maize = np.squeeze(maize_batch, axis=0)
    print(maize.shape)
    plt.imshow(maize)

In [ ]:
plt.imshow(conv_maize)

**Here is the example of visualizing probabilities of predictions of a particular image.**

**Let's now build a very simple model to witness the filtering ability of a neural network.**

**Please note that this model (simple_model) should not be used to make trustworthy predictions. This is just for the purpose of exercise.**

In [ ]:
simple_model = tf.keras.models.Sequential()
simple_model.add(tf.keras.layers.Conv2D(1,3,3,input_shape=maize.shape)) # 3x3 kernel

In [ ]:
# Function to show the mask of the image (aka how the model sees the image)
def visualize_color(simple_model, maize):
    maize_batch = np.expand_dims(maize, axis=0)
    conv_maize2 = simple_model.predict(maize_batch)
    conv_maize2 = np.squeeze(conv_maize2, axis=0)
    
    print(conv_maize2.shape)
    conv_maize2 = conv_maize2.reshape(conv_maize2.shape[:2])
    print(conv_maize2.shape)
    plt.imshow(conv_maize2)

In [ ]:
visualize_color(simple_model, maize)

**Okay, our model can see the pattern of leaves and can make some predictions based on it. That's good.**

# 4. Conclusion

We have built a **CNN-model** to predict the class of a plant, which works quite well. (Increasing **number of epochs** and/or **adding layers** to a model can even increase the performance.

**CNN + Maxpooling + Global pooling + Dense** is a good combination for image classification.

If you want to understand how your neural network works and what features of images it considers important, you can build a function similar to **visualize_color** (see above).